In [12]:
from sagemaker.inputs import FileSystemInput

subnets = ["subnet-0160e760b20ce0ad6"]  # Should be same as Subnet used for FSx. Example: subnet-01aXXXX
security_group_ids = [
    "sg-0b2c3997c550d07b6"
]  # Should be same as Security group used for EFS. sg-075ZZZZZZ
file_system_id = "fs-0a9ebc472fa7d5342"  # EFS file system ID with your training dataset. Example: 'fs-0bYYYYYY'

file_system_directory_path = "/"
file_system_access_mode = "ro"

file_system_type = "EFS"

train_fs = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path,
    file_system_access_mode=file_system_access_mode,
)

In [13]:
import wandb
wandb.login()
settings = wandb.setup().settings
current_api_key = wandb.wandb_lib.apikey.api_key(settings=settings)

In [14]:
import sagemaker
import boto3
boto3_session = boto3.Session(profile_name="cornell")
session = sagemaker.Session(boto_session=boto3_session)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [15]:
from sagemaker.pytorch import PyTorch

# docker_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker"
instance_type = "ml.p3.2xlarge"
instance_count = 1
# This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
job_name = "clip-forge-autoencoder"
print("Job name: ", job_name)

estimator = PyTorch(
    base_job_name=job_name,
    source_dir=".",
    entry_point="train_autoencoder.py",  #  the entry point that launches the training script with options
    role="arn:aws:iam::870747888580:role/SageMakerTrainingRole",
    sagemaker_session=session,
    framework_version="1.13.1",
    py_version="py39",
    instance_count=instance_count,
    instance_type=instance_type,
    # subnets=subnets,
    # security_group_ids=security_group_ids,
    debugger_hook_config=False,
    # Training using SMDataParallel Distributed Training Framework
    # distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
    environment={"WANDB_API_KEY": current_api_key},
    input_mode="FastFile",
)

Job name:  clip-forge-autoencoder


In [16]:
# estimator.fit({ "train": train_fs })
estimator.fit({ "train": "s3://cornell-mfd64/text2building/shapenet" })

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: clip-forge-autoencoder-2023-02-26-02-10-55-560


2023-02-26 02:11:17 Starting - Starting the training job...
2023-02-26 02:11:42 Starting - Preparing the instances for training.........
2023-02-26 02:12:52 Downloading - Downloading input data...............
2023-02-26 02:15:32 Training - Downloading the training image.............